In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%cd /content/gdrive/My Drive/fin_subcate/

/content/gdrive/My Drive/fin_subcate


In [ ]:
!ls

test_fin_cate.csv  train_fin_cate.csv


### Import train, test Tech Group data of Technology Services department

### Dataframe to Dataset

In [10]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 

In [3]:
import pandas as pd
train = pd.read_csv('train_fin_subcate.csv')
test = pd.read_csv('test_fin_subcate.csv')

In [4]:
# Calculate the sum of each row to determine the number of labels per description
rowsums = train.iloc[:, 1:].sum(axis=1)

# Count the number of descriptions without any label
no_label_count = (rowsums == 0).sum()

# Filter the DataFrame to exclude rows with no labels
train_df_new = train[rowsums != 0]

print(f"Number of descriptions without any label: {no_label_count}")
print(f"Shape of original DataFrame: {train.shape}")
print(f"Shape of filtered DataFrame: {train_df_new.shape}")

Number of descriptions without any label: 53
Shape of original DataFrame: (4092, 21)
Shape of filtered DataFrame: (4039, 21)


In [5]:
# Calculate the sum of each row to determine the number of labels per description
rowsums = test.iloc[:, 1:].sum(axis=1)

# Count the number of descriptions without any label
no_label_count = (rowsums == 0).sum()

# Filter the DataFrame to exclude rows with no labels
test_df_new = test[rowsums != 0]

print(f"Number of descriptions without any label: {no_label_count}")
print(f"Shape of original DataFrame: {test.shape}")
print(f"Shape of filtered DataFrame: {test_df_new.shape}")

Number of descriptions without any label: 13
Shape of original DataFrame: (1024, 21)
Shape of filtered DataFrame: (1011, 21)


In [6]:
train = train_df_new
test = test_df_new

In [7]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)

In [ ]:
train_dataset

Dataset({
    features: ['Description', 'Billing - Spaces', 'ACCOUNTS PAYABLE', 'TREASURY', '__index_level_0__'],
    num_rows: 4079
})

### Label Encoding

In [8]:
import os
def department_label_encoding(train_dataset):
    labels = [col for col in train_dataset.column_names if col != 'Description']
    labels = labels[:-1]
    label2id = {label: idx for idx, label in enumerate(labels)}
    id2label = {idx: label for label, idx in label2id.items()}
    print(label2id, id2label)
    return label2id, id2label

In [9]:
label2id, id2label = department_label_encoding(train_dataset)

{'Other Request': 0, 'Below 20k invoices (No PO)': 1, 'Client Cash advance': 2, 'Client Refund': 3, 'Confidential Payments Requests': 4, 'Complicated Report': 5, 'FORMS (Reimbursement, PCF, Credit Card Liquidation)': 6, 'Internal & External reimb request': 7, 'Petty Cash Request': 8, 'PO invoices': 9, 'Recurring bills (rentals, utilities, telco, etc.)': 10, 'Simple Report': 11, 'Urgent requests (Business permits, other taxes etc.)': 12, 'Bank Request': 13, 'Cash Advance': 14, 'Client Request': 15, 'Government Statutory': 16, 'Last Pay': 17, 'Operations': 18, 'Taxes / BOC / LGU Payment': 19} {0: 'Other Request', 1: 'Below 20k invoices (No PO)', 2: 'Client Cash advance', 3: 'Client Refund', 4: 'Confidential Payments Requests', 5: 'Complicated Report', 6: 'FORMS (Reimbursement, PCF, Credit Card Liquidation)', 7: 'Internal & External reimb request', 8: 'Petty Cash Request', 9: 'PO invoices', 10: 'Recurring bills (rentals, utilities, telco, etc.)', 11: 'Simple Report', 12: 'Urgent requests 

### import model

In [11]:
!pip install protobuf

In [10]:
from transformers import AutoTokenizer
model_path = 'microsoft/deberta-v3-small'
tokenizer = AutoTokenizer.from_pretrained(model_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


### Tokenization Node

In [11]:
departments = [col for col in train_dataset.column_names if col != 'Description']
departments = departments[:-1]
departments

['Other Request',
 'Below 20k invoices (No PO)',
 'Client Cash advance',
 'Client Refund',
 'Confidential Payments Requests',
 'Complicated Report',
 'FORMS (Reimbursement, PCF, Credit Card Liquidation)',
 'Internal & External reimb request',
 'Petty Cash Request',
 'PO invoices',
 'Recurring bills (rentals, utilities, telco, etc.)',
 'Simple Report',
 'Urgent requests (Business permits, other taxes etc.)',
 'Bank Request',
 'Cash Advance',
 'Client Request',
 'Government Statutory',
 'Last Pay',
 'Operations',
 'Taxes / BOC / LGU Payment']

In [12]:
def preprocess_function(example):
  text = example["Description"]
  labels = [0. for _ in range(len(departments))]
  for department in departments:
      if example[department] == 1:
          label_id = label2id[department]
          labels[label_id] = 1.

  example = tokenizer(text, truncation=True, padding='max_length', max_length=512)
  example['labels'] = labels
  return example

In [13]:
tokenized_train_dataset = train_dataset.map(preprocess_function)
tokenized_test_dataset = test_dataset.map(preprocess_function)

Map:   0%|          | 0/4039 [00:00<?, ? examples/s]

Map:   0%|          | 0/1011 [00:00<?, ? examples/s]

### Train Model

In [12]:
!pip install huggingface_hub
from huggingface_hub import notebook_login

notebook_login()

In [15]:
!pip3 install transformers[torch] accelerate evaluate sentencepiece accelerate

In [14]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = sigmoid(predictions)
    predictions = (predictions > 0.5).astype(int)

    # Compute metrics
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='micro')
    precision = precision_score(labels, predictions, average='micro')
    recall = recall_score(labels, predictions, average='micro')

    return {"accuracy": accuracy, "f1": f1, "precision": precision, "recall": recall}

In [15]:
len(label2id)

20

In [16]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding

model = AutoModelForSequenceClassification.from_pretrained(
   model_path, num_labels=len(label2id),
           id2label=id2label, label2id=label2id,
                       problem_type = "multi_label_classification", ignore_mismatched_sizes=True)

pytorch_model.bin:   0%|          | 0.00/286M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
training_args = TrainingArguments(
    output_dir='./fin_subcate',
    eval_strategy="epoch",
    learning_rate=2e-5,  # Adjusted learning rate
    per_device_train_batch_size=64,  # Increased batch size
    per_device_eval_batch_size=128,  # Increased eval batch size
    num_train_epochs=7,  # Increased epochs
    weight_decay=0.01,
)

In [19]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
# Initialize the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [20]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.187539,0.000000,0.000000,0.000000,0.000000
2,No log,0.117894,0.513353,0.657795,0.915344,0.513353
3,No log,0.093072,0.512364,0.667956,0.959259,0.512364
4,No log,0.079767,0.623145,0.734266,0.893617,0.623145
5,No log,0.071748,0.650841,0.755020,0.898907,0.650841
6,No log,0.068359,0.674580,0.762864,0.877735,0.674580
7,No log,0.067128,0.682493,0.767093,0.875635,0.682493


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=448, training_loss=0.13975439752851213, metrics={'train_runtime': 489.5948, 'train_samples_per_second': 57.748, 'train_steps_per_second': 0.915, 'total_flos': 3746586413101056.0, 'train_loss': 0.13975439752851213, 'epoch': 7.0})

In [21]:
trainer.evaluate()

{'eval_loss': 0.0671280026435852,
 'eval_accuracy': 0.6824925816023739,
 'eval_f1': 0.7670928293496386,
 'eval_precision': 0.8756345177664975,
 'eval_recall': 0.6824925816023739,
 'eval_runtime': 5.6956,
 'eval_samples_per_second': 177.505,
 'eval_steps_per_second': 1.405,
 'epoch': 7.0}

In [22]:
trainer.push_to_hub('fin_subcate')

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

events.out.tfevents.1721374382.caf7575f2689.10048.0:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

events.out.tfevents.1721374877.caf7575f2689.10048.1:   0%|          | 0.00/560 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/568M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/tangminhanh/fin_subcate/commit/1c1e357ad29ecdad6dccb484cda039c5af58ddda', commit_message='fin_subcate', commit_description='', oid='1c1e357ad29ecdad6dccb484cda039c5af58ddda', pr_url=None, pr_revision=None, pr_num=None)